In [6]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Subset
import numpy as np
from tqdm import tqdm
import pandas as pd
import random
import json

device = torch.device("cuda")

class RandomGaussianBlur:
    def __init__(self, kernel_size=3, probability=0.5):
        self.kernel_size = kernel_size
        self.probability = probability
        self.gaussian_blur = transforms.GaussianBlur(self.kernel_size)

    def __call__(self, img):
        if random.random() < self.probability:
            return self.gaussian_blur(img)
        return img

os.chdir('/home/kdoherty/spurge/data_release')

train_dir = './data/crop_39/train'

seed = 0
batch_size = 32

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

df = pd.read_csv('./results/best_lr.csv')
best_row = df.loc[df['accuracy'].idxmax()]
learning_rate = best_row['lr']

with open('./results/best_augs.json', 'r') as file:
    augs = json.load(file)

with open('./results/best_epoch.json', 'r') as file:
    n_epochs = json.load(file)['n_epochs']

gaussian_blur = augs['gaussian_blur']
flip_horizontal = augs['flip_horizontal']
flip_vertical = augs['flip_vertical']
brightness = augs['brightness']
contrast = augs['contrast']
saturation = augs['saturation']
hue = augs['hue']
rotation = augs['rotation']

stats = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_list = [transforms.ToTensor(), stats]

if gaussian_blur:
    transform_list.insert(0, RandomGaussianBlur())
if flip_horizontal:
    transform_list.insert(0, transforms.RandomHorizontalFlip())
if flip_vertical:
    transform_list.insert(0, transforms.RandomVerticalFlip())

transform_list.insert(0, transforms.ColorJitter(hue=hue, contrast=contrast, brightness=brightness, saturation=saturation))
transform_list.insert(0, transforms.RandomRotation(rotation))

data_transforms = {
    'train': transforms.Compose(transform_list),
    'val': transforms.Compose([
        transforms.ToTensor(),
        stats
    ])
}

full_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])

# Create the data loaders
train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

# Load pre-trained resnet50 model + higher level layers
model = models.resnet50(pretrained=True)

# Change the last layer to have 1 output
num_ftrs = model.fc.in_features

# Modify fc layers for binary classification
model.fc = nn.Sequential(
  nn.Linear(num_ftrs, 1),
  )

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss().to(device)

model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

epochs = range(n_epochs)
epoch_accs = []
epoch_losses = []

with tqdm(total=n_epochs*len(train_loader), unit="batch", desc="Training Progress") as pbar:
    for epoch in epochs:
        model.train()
        running_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images).squeeze(1)  # Remove only the second dimension if it's size 1
            loss = criterion(output, labels.float())  # No need to squeeze again
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            pbar.update(1)
        
        train_loss = running_loss / len(train_loader)
        
        # Validate the model
        model.eval()
        running_loss = 0
        running_acc = 0
        
        pbar.set_postfix({'Epoch': epoch, 
                          'Training Loss': f'{train_loss:.3f}',
                         })

/home/kdoherty/anaconda3/envs/data-release/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kdoherty/anaconda3/envs/data-release/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Training Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [00:38<00:00,  9.56batch/s, Epoch=13, Training Loss=0.597]


In [7]:
test_dir = './data/crop_39/val'

# Apply the same normalization stats but not the augmentation transformations for the test set
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    stats
])

# Load the test data
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Create the test DataLoader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Function to evaluate the model on the test set
def evaluate(model, loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    test_acc = 0
    with torch.no_grad():  # No gradients required for evaluation
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images).squeeze(1)
            loss = criterion(output, labels.float())
            test_loss += loss.item()
            acc = binary_accuracy(output, labels)
            test_acc += acc.item()
    
    test_loss /= len(loader)
    test_acc /= len(loader)
    return test_loss, test_acc

# Load the model weights if necessary, for example:
# model.load_state_dict(torch.load('path_to_your_saved_model.pth'))

# Calculate loss and accuracy on the test set
test_loss, test_accuracy = evaluate(model, test_loader)

print(f"Test Loss: {test_loss:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")

Test Loss: 0.452
Test Accuracy: 0.820


In [ ]:
with open('./results/test_perormance.json', 'w') as f:
    json.dump({'loss':test_loss,'accuracy':test_accuracy}, f)